<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/MLP/Salvando_Recarregando_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Salvando e Carregando Modelos Treinados

In [1]:
!nvidia-smi

Thu Jan 17 15:43:34 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    58W / 149W |   1303MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Definindo um Grafo Multilayer Perceptron 

In [0]:
import tensorflow as tf


##########################
### Definindo o Modelo
##########################

# Função que define uma camada
def fc_layer(input_tensor, 
             n_output_units, 
             name, 
             activation_fn = None, 
             seed = None, 
             weight_params = None, 
             bias_params = None):

    with tf.variable_scope(name):

        if weight_params is not None:
            weights = tf.constant(weight_params, name = 'weights', dtype = tf.float32)
        else:
            weights = tf.Variable(tf.truncated_normal(
                shape=[input_tensor.get_shape().as_list()[-1], n_output_units],
                    mean = 0.0,
                    stddev = 0.1,
                    dtype = tf.float32,
                    seed = seed),
                name = 'weights',)

        if bias_params is not None:
            biases = tf.constant(bias_params, name = 'biases', dtype=tf.float32)

        else:
            biases = tf.Variable(tf.zeros(shape = [n_output_units]),
                                 name = 'biases', 
                                 dtype = tf.float32)

        act = tf.matmul(input_tensor, weights) + biases

        if activation_fn is not None:
            act = activation_fn(act)

    return act

# Função que define o grafo
def mlp_graph(n_input = 784, 
              n_classes = 10, 
              n_hidden_1 = 128, 
              n_hidden_2 = 256,
              learning_rate = 0.1,
              fixed_params = None):
    
    # Carregando pesos e bias de arquivos NumPy
    if not fixed_params:
        var_names = ['fc1/weights:0', 'fc1/biases:0',
                     'fc2/weights:0', 'fc2/biases:0',
                     'logits/weights:0', 'logits/biases:0',]
        
        fixed_params = {v: None for v in var_names}
        found_params = False
    else:
        found_params = True
    
    # Input data
    tf_x = tf.placeholder(tf.float32, [None, n_input], name = 'features')
    tf_y = tf.placeholder(tf.int32, [None], name = 'targets')
    tf_y_onehot = tf.one_hot(tf_y, depth = n_classes, name = 'onehot_targets')

    # Multilayer perceptron
    fc1 = fc_layer(input_tensor = tf_x, 
                   n_output_units = n_hidden_1, 
                   name = 'fc1',
                   weight_params = fixed_params['fc1/weights:0'], 
                   bias_params = fixed_params['fc1/biases:0'],
                   activation_fn = tf.nn.relu)

    fc2 = fc_layer(input_tensor = fc1, 
                   n_output_units = n_hidden_2, 
                   name = 'fc2',
                   weight_params = fixed_params['fc2/weights:0'], 
                   bias_params = fixed_params['fc2/biases:0'],
                   activation_fn = tf.nn.relu)
    
    logits = fc_layer(input_tensor = fc2, 
                      n_output_units = n_classes, 
                      name = 'logits',
                      weight_params = fixed_params['logits/weights:0'], 
                      bias_params = fixed_params['logits/biases:0'],
                      activation_fn = tf.nn.relu)
    
    # Loss e optimizer
    ### Somente necessário se nenhum parâmetro existente for encontrado
    ### e um grafo treinável deve ser inicializado
    if not found_params:
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=tf_y_onehot)
        cost = tf.reduce_mean(loss, name='cost')
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=learning_rate)
        train = optimizer.minimize(cost, name='train')

    # Previsões
    probabilities = tf.nn.softmax(logits, name = 'probabilities')
    labels = tf.cast(tf.argmax(logits, 1), tf.int32, name = 'labels')
    
    correct_prediction = tf.equal(labels, tf_y, name='correct_predictions')
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy')

## Treinando e Salvando o Modelo Multilayer Perceptron

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

##########################
### Configurações
##########################

# Hiperparâmetros
learning_rate = 0.1
training_epochs = 10
batch_size = 64

##########################
### Definição do Grafo
##########################

g = tf.Graph()
with g.as_default():
    mlp_graph()

##########################
### DATASET
##########################
file = '.'
mnist = input_data.read_data_sets(file, one_hot = False)

############################
### Treinamento e Avaliação
############################

with tf.Session(graph = g) as sess:
    sess.run(tf.global_variables_initializer())
    saver0 = tf.train.Saver()
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = mnist.train.num_examples // batch_size

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run(['train', 'cost:0'], feed_dict = {'features:0': batch_x,
                                                            'targets:0': batch_y})
            avg_cost += c
        
        train_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.train.images,
                                                        'targets:0': mnist.train.labels})
        
        valid_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.validation.images,
                                                        'targets:0': mnist.validation.labels})  
        
        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Acurácia em Treino/Validação: %.3f/%.3f" % (train_acc, valid_acc))
        
    test_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.test.images,
                                                   'targets:0': mnist.test.labels})
    print('Acurácia em Teste: %.3f' % test_acc)
    
    ##################################
    ### Salvando o Modelo Treinado ###
    ##################################
    saver0.save(sess, save_path = './mlp')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Epoch: 001 | AvgCost: 0.576 | Acurácia em Treino/Validação: 0.855/0.846
Epoch: 002 | AvgCost: 0.385 | Acurácia em Treino/Validação: 0.870/0.858
Epoch: 003 | AvgCost: 0.340 | Acurácia em Treino/Validação: 0.867/0.856
Epoch: 004 | AvgCost: 0.238 | Acurácia em Treino/Validação: 0.970/0.964
Epoch: 005 | AvgCost: 0.085 | Acurácia em Treino/Validação: 0.978/0.972
Epoch: 006 | AvgCost: 0.068 | Acurácia

## Recarregando o Modelo de Arquivos Meta e Checkpoint 

In [7]:
!ls -ilah mlp.meta

15861705 -rw-r--r-- 1 root root 36K Jan 17 15:48 mlp.meta


In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

file = '.'
mnist = input_data.read_data_sets(file, one_hot = False)

with tf.Session() as sess:
    
    saver1 = tf.train.import_meta_graph('./mlp.meta')
    saver1.restore(sess, save_path = './mlp')
    
    test_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.test.images,
                                                   'targets:0': mnist.test.labels})
    print('Acurácia em Teste: %.3f' % test_acc)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from ./mlp
Acurácia em Teste: 0.976


## Trabalhando com Arquivos NumPy e Criando Grafos Não-Treinados

### Exportando os Parâmetros do Modelo Para Arquivos NumPy NPZ 

In [9]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()
with tf.Session() as sess:

    saver1 = tf.train.import_meta_graph('./mlp.meta')
    saver1.restore(sess, save_path='./mlp')
    
    var_names = [v.name for v in 
                 tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)]
    
    params = {}
    print('Variáveis encontradas:')
    for v in var_names:
        print(v)
        
        ary = sess.run(v)
        params[v] = ary
        
    np.savez('mlp', **params)

INFO:tensorflow:Restoring parameters from ./mlp
Variáveis encontradas:
fc1/weights:0
fc1/biases:0
fc2/weights:0
fc2/biases:0
logits/weights:0
logits/biases:0


### Carregando Arquivos NumPy .npz em `mlp_graph`

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

###########################
### Carregando Dados e Parâmetros
###########################

file = '.'
mnist = input_data.read_data_sets(file, one_hot=False)
param_dict = np.load('mlp.npz')

##########################
### Definição do Grafo
##########################


g = tf.Graph()
with g.as_default():
    
    mlp_graph(fixed_params = param_dict)

with tf.Session(graph=g) as sess:
    
    test_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.test.images,'targets:0': mnist.test.labels})
    print('Acurácia em Teste: %.3f' % test_acc)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Acurácia em Teste: 0.976
